# iohub Format Conversion & Benchmarking

## Overview

This notebook benchmarks format conversion using [iohub](https://github.com/czbiohub-sf/iohub):
- 🔄 **CryoET Data** → TIFF → OME-Zarr (v2 & v3)
- ⚡ **Conversion speed** comparison
- 💾 **Storage efficiency** analysis
- 📊 **Data integrity** validation
- 🔬 **Metadata preservation** testing

**iohub Features:**
- Unified interface for bioimaging formats
- OME-NGFF v0.4 and v0.5 support
- Efficient TIFF ↔ OME-Zarr conversion
- Automatic metadata construction and validation

**Use Cases:**
- Converting legacy TIFF data to modern OME-Zarr
- Benchmarking different zarr versions
- Validating conversion pipelines

In [1]:
# Verify environment
import sys

print(f"Python: {sys.version}")
print(f"Executable: {sys.executable}")
if "3.13" in sys.version and "venv" in sys.executable:
    print("✓ Correct environment!")
else:
    print("⚠️  Please select 'Python 3.13 (zarr-benchmarks)' kernel")

Python: 3.13.5 (main, Jun 11 2025, 15:36:57) [Clang 17.0.0 (clang-1700.0.13.3)]
Executable: /Users/mkothari/zarr-benchmarks/venv/bin/python
✓ Correct environment!


In [2]:
# Import libraries
import os
import pathlib
import time
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import s3fs
import zarr
from cryoet_data_portal import Client, Dataset
from iohub import save_tiff
from iohub.ngff import open_ome_zarr
from skimage.metrics import mean_squared_error as mse
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim

from zarr_benchmarks import utils

warnings.filterwarnings("ignore")

# Enable Zarr v3 experimental API for testing
os.environ["ZARR_V3_EXPERIMENTAL_API"] = "1"

print("✓ iohub available")
print(f"✓ Zarr version: {zarr.__version__}")
print("✓ All imports successful")

ImportError: cannot import name 'save_tiff' from 'iohub' (/Users/mkothari/zarr-benchmarks/venv/lib/python3.13/site-packages/iohub/__init__.py)

## Configuration

In [ ]:
# Test configuration
DOWNLOAD_SIZE = 256  # Cube size for testing

# Conversion paths to test
CONVERSION_PATHS = [
    "Original → TIFF",
    "TIFF → OME-Zarr v2",
    "TIFF → OME-Zarr v3",
    "Original → OME-Zarr v2 (direct)",
    "Original → OME-Zarr v3 (direct)",
]

# Compression for zarr conversion
ZARR_CODEC = "blosc_zstd"
ZARR_LEVEL = 5
CHUNK_SIZE = 128

print("Configuration:")
print(f"  Data size: {DOWNLOAD_SIZE}³ = {(DOWNLOAD_SIZE**3 * 4) / (1024**2):.1f} MB")
print(f"  Conversion paths: {len(CONVERSION_PATHS)}")
print(f"  Zarr codec: {ZARR_CODEC} level {ZARR_LEVEL}")
print(f"  Chunk size: {CHUNK_SIZE}³")

## 1. Download CryoET Data

In [ ]:
# Connect to CryoET Portal and download data
print("Connecting to CryoET Data Portal...")
client = Client()
dataset = Dataset.get_by_id(client, 10445)

print(f"✓ Dataset: {dataset.title}")

# Find suitable tomogram
runs = list(dataset.runs)
selected_tomo = None

for run in runs:
    for tomo in list(run.tomograms):
        if (
            tomo.size_x >= DOWNLOAD_SIZE
            and tomo.size_y >= DOWNLOAD_SIZE
            and tomo.size_z >= DOWNLOAD_SIZE
        ):
            selected_tomo = tomo
            break
    if selected_tomo:
        break

print(f"✓ Selected: {selected_tomo.name}")
print(
    f"  Size: {selected_tomo.size_x} × {selected_tomo.size_y} × {selected_tomo.size_z}"
)

# Download centered cube
s3 = s3fs.S3FileSystem(anon=True)
zarr_path = selected_tomo.s3_omezarr_dir.replace("s3://", "")
store = s3fs.S3Map(root=zarr_path, s3=s3, check=False)
zarr_group = zarr.open(store, mode="r")
zarr_array = zarr_group["0"]

z_c = zarr_array.shape[0] // 2
y_c = zarr_array.shape[1] // 2
x_c = zarr_array.shape[2] // 2

z_start = max(0, z_c - DOWNLOAD_SIZE // 2)
z_end = z_start + DOWNLOAD_SIZE
y_start = max(0, y_c - DOWNLOAD_SIZE // 2)
y_end = y_start + DOWNLOAD_SIZE
x_start = max(0, x_c - DOWNLOAD_SIZE // 2)
x_end = x_start + DOWNLOAD_SIZE

print(f"\nDownloading {DOWNLOAD_SIZE}³ cube...")
reference_data = np.array(zarr_array[z_start:z_end, y_start:y_end, x_start:x_end])

print("✓ Downloaded")
print(f"  Shape: {reference_data.shape}")
print(f"  Size: {reference_data.nbytes / (1024**2):.2f} MB")
print(f"  Dtype: {reference_data.dtype}")
print(f"  Range: [{reference_data.min():.3f}, {reference_data.max():.3f}]")

## 2. Conversion Benchmarks

In [ ]:
# Helper functions
def calculate_metrics(original, converted):
    """Calculate SSIM, PSNR, MSE"""
    try:
        orig_norm = (original - original.min()) / (
            original.max() - original.min() + 1e-10
        )
        conv_norm = (converted - converted.min()) / (
            converted.max() - converted.min() + 1e-10
        )

        mid = original.shape[0] // 2
        ssim_val = ssim(orig_norm[mid], conv_norm[mid], data_range=1.0)

        data_range = original.max() - original.min()
        psnr_val = psnr(original, converted, data_range=data_range)
        mse_val = mse(original, converted)

        return ssim_val, psnr_val, mse_val
    except Exception:
        return None, None, None


def get_size_mb(path):
    """Get total size of file/directory in MB"""
    if pathlib.Path(path).is_file():
        return pathlib.Path(path).stat().st_size / (1024**2)
    else:
        return utils.get_directory_size(path) / (1024**2)


def count_files(path):
    """Count files in directory"""
    if pathlib.Path(path).is_file():
        return 1
    total = 0
    for p in pathlib.Path(path).rglob("*"):
        if p.is_file():
            total += 1
    return total


print("✓ Helper functions ready")

In [ ]:
# Setup output directory
output_dir = pathlib.Path("data/output/iohub_conversion")
output_dir.mkdir(parents=True, exist_ok=True)

# Store results
results = []

print(f"✓ Output directory: {output_dir}")

### 2.1 Convert to TIFF

In [ ]:
print("=" * 70)
print("[1/5] Original → TIFF")
print("=" * 70)

tiff_path = output_dir / "cryoet_data.tiff"

try:
    # Convert to TIFF using iohub
    t0 = time.time()
    save_tiff(
        tiff_path,
        reference_data,
        axes="ZYX",
        compression="zlib",
        compressionargs={"level": ZARR_LEVEL},
    )
    conversion_time = time.time() - t0

    # Read back to verify
    t0 = time.time()
    import tifffile

    tiff_data = tifffile.imread(tiff_path)
    read_time = time.time() - t0

    # Calculate metrics
    size_mb = get_size_mb(tiff_path)
    file_count = count_files(tiff_path)
    compression_ratio = reference_data.nbytes / (size_mb * 1024**2)
    ssim_val, psnr_val, mse_val = calculate_metrics(reference_data, tiff_data)

    results.append(
        {
            "conversion_path": "Original → TIFF",
            "format": "TIFF",
            "conversion_time": conversion_time,
            "read_time": read_time,
            "size_mb": size_mb,
            "file_count": file_count,
            "compression_ratio": compression_ratio,
            "ssim": ssim_val,
            "psnr": psnr_val,
            "mse": mse_val,
            "success": True,
        }
    )

    print("✓ Success")
    print(f"  Conversion time: {conversion_time:.3f}s")
    print(f"  Read time: {read_time:.3f}s")
    print(f"  Size: {size_mb:.2f} MB ({compression_ratio:.2f}× compression)")
    print(f"  SSIM: {ssim_val:.6f}")

except Exception as e:
    print(f"✗ Failed: {e}")
    results.append(
        {
            "conversion_path": "Original → TIFF",
            "format": "TIFF",
            "success": False,
            "error": str(e),
        }
    )
    tiff_data = None

### 2.2 TIFF → OME-Zarr (v2 & v3)

In [3]:
print("\n" + "=" * 70)
print("[2/5] TIFF → OME-Zarr v2")
print("=" * 70)

if tiff_data is not None:
    try:
        zarr_v2_from_tiff = output_dir / "from_tiff_v2.zarr"
        utils.remove_output_dir(zarr_v2_from_tiff)

        # Convert using iohub
        t0 = time.time()
        with open_ome_zarr(
            zarr_v2_from_tiff,
            layout="hcs",
            mode="w",
            channel_names=["cryoet"],
            zarr_version=2,
        ) as dataset:
            pos = dataset.create_position("0", "0", "0")
            pos["0"][:] = tiff_data[np.newaxis, np.newaxis, ...]  # Add T, C dims

        conversion_time = time.time() - t0

        # Read back
        t0 = time.time()
        with open_ome_zarr(zarr_v2_from_tiff, mode="r") as dataset:
            pos = dataset["0/0/0"]
            zarr_data = np.array(pos["0"][0, 0, ...])
        read_time = time.time() - t0

        # Calculate metrics
        size_mb = get_size_mb(zarr_v2_from_tiff)
        file_count = count_files(zarr_v2_from_tiff)
        compression_ratio = reference_data.nbytes / (size_mb * 1024**2)
        ssim_val, psnr_val, mse_val = calculate_metrics(reference_data, zarr_data)

        results.append(
            {
                "conversion_path": "TIFF → OME-Zarr v2",
                "format": "OME-Zarr v2",
                "conversion_time": conversion_time,
                "read_time": read_time,
                "size_mb": size_mb,
                "file_count": file_count,
                "compression_ratio": compression_ratio,
                "ssim": ssim_val,
                "psnr": psnr_val,
                "mse": mse_val,
                "success": True,
            }
        )

        print("✓ Success")
        print(f"  Conversion time: {conversion_time:.3f}s")
        print(f"  Read time: {read_time:.3f}s")
        print(f"  Size: {size_mb:.2f} MB ({compression_ratio:.2f}× compression)")
        print(f"  Files: {file_count}")
        print(f"  SSIM: {ssim_val:.6f}")

    except Exception as e:
        print(f"✗ Failed: {e}")
        results.append(
            {
                "conversion_path": "TIFF → OME-Zarr v2",
                "format": "OME-Zarr v2",
                "success": False,
                "error": str(e),
            }
        )
else:
    print("⊘ Skipped (TIFF conversion failed)")


[2/5] TIFF → OME-Zarr v2


NameError: name 'tiff_data' is not defined

In [4]:
print("\n" + "=" * 70)
print("[3/5] TIFF → OME-Zarr v3")
print("=" * 70)

if tiff_data is not None:
    try:
        zarr_v3_from_tiff = output_dir / "from_tiff_v3.zarr"
        utils.remove_output_dir(zarr_v3_from_tiff)

        # Convert using iohub
        t0 = time.time()
        with open_ome_zarr(
            zarr_v3_from_tiff,
            layout="hcs",
            mode="w",
            channel_names=["cryoet"],
            zarr_version=3,
        ) as dataset:
            pos = dataset.create_position("0", "0", "0")
            pos["0"][:] = tiff_data[np.newaxis, np.newaxis, ...]

        conversion_time = time.time() - t0

        # Read back
        t0 = time.time()
        with open_ome_zarr(zarr_v3_from_tiff, mode="r") as dataset:
            pos = dataset["0/0/0"]
            zarr_data = np.array(pos["0"][0, 0, ...])
        read_time = time.time() - t0

        # Calculate metrics
        size_mb = get_size_mb(zarr_v3_from_tiff)
        file_count = count_files(zarr_v3_from_tiff)
        compression_ratio = reference_data.nbytes / (size_mb * 1024**2)
        ssim_val, psnr_val, mse_val = calculate_metrics(reference_data, zarr_data)

        results.append(
            {
                "conversion_path": "TIFF → OME-Zarr v3",
                "format": "OME-Zarr v3",
                "conversion_time": conversion_time,
                "read_time": read_time,
                "size_mb": size_mb,
                "file_count": file_count,
                "compression_ratio": compression_ratio,
                "ssim": ssim_val,
                "psnr": psnr_val,
                "mse": mse_val,
                "success": True,
            }
        )

        print("✓ Success")
        print(f"  Conversion time: {conversion_time:.3f}s")
        print(f"  Read time: {read_time:.3f}s")
        print(f"  Size: {size_mb:.2f} MB ({compression_ratio:.2f}× compression)")
        print(f"  Files: {file_count}")
        print(f"  SSIM: {ssim_val:.6f}")

    except Exception as e:
        print(f"✗ Failed: {e}")
        results.append(
            {
                "conversion_path": "TIFF → OME-Zarr v3",
                "format": "OME-Zarr v3",
                "success": False,
                "error": str(e),
            }
        )
else:
    print("⊘ Skipped (TIFF conversion failed)")


[3/5] TIFF → OME-Zarr v3


NameError: name 'tiff_data' is not defined

### 2.3 Direct OME-Zarr Conversion (v2 & v3)

In [5]:
print("\n" + "=" * 70)
print("[4/5] Original → OME-Zarr v2 (direct)")
print("=" * 70)

try:
    zarr_v2_direct = output_dir / "direct_v2.zarr"
    utils.remove_output_dir(zarr_v2_direct)

    # Direct conversion using iohub
    t0 = time.time()
    with open_ome_zarr(
        zarr_v2_direct, layout="hcs", mode="w", channel_names=["cryoet"], zarr_version=2
    ) as dataset:
        pos = dataset.create_position("0", "0", "0")
        pos["0"][:] = reference_data[np.newaxis, np.newaxis, ...]

    conversion_time = time.time() - t0

    # Read back
    t0 = time.time()
    with open_ome_zarr(zarr_v2_direct, mode="r") as dataset:
        pos = dataset["0/0/0"]
        zarr_data = np.array(pos["0"][0, 0, ...])
    read_time = time.time() - t0

    # Calculate metrics
    size_mb = get_size_mb(zarr_v2_direct)
    file_count = count_files(zarr_v2_direct)
    compression_ratio = reference_data.nbytes / (size_mb * 1024**2)
    ssim_val, psnr_val, mse_val = calculate_metrics(reference_data, zarr_data)

    results.append(
        {
            "conversion_path": "Original → OME-Zarr v2 (direct)",
            "format": "OME-Zarr v2",
            "conversion_time": conversion_time,
            "read_time": read_time,
            "size_mb": size_mb,
            "file_count": file_count,
            "compression_ratio": compression_ratio,
            "ssim": ssim_val,
            "psnr": psnr_val,
            "mse": mse_val,
            "success": True,
        }
    )

    print("✓ Success")
    print(f"  Conversion time: {conversion_time:.3f}s")
    print(f"  Read time: {read_time:.3f}s")
    print(f"  Size: {size_mb:.2f} MB ({compression_ratio:.2f}× compression)")
    print(f"  Files: {file_count}")
    print(f"  SSIM: {ssim_val:.6f}")

except Exception as e:
    print(f"✗ Failed: {e}")
    results.append(
        {
            "conversion_path": "Original → OME-Zarr v2 (direct)",
            "format": "OME-Zarr v2",
            "success": False,
            "error": str(e),
        }
    )


[4/5] Original → OME-Zarr v2 (direct)
✗ Failed: name 'output_dir' is not defined


NameError: name 'results' is not defined

In [6]:
print("\n" + "=" * 70)
print("[5/5] Original → OME-Zarr v3 (direct)")
print("=" * 70)

try:
    zarr_v3_direct = output_dir / "direct_v3.zarr"
    utils.remove_output_dir(zarr_v3_direct)

    # Direct conversion using iohub
    t0 = time.time()
    with open_ome_zarr(
        zarr_v3_direct, layout="hcs", mode="w", channel_names=["cryoet"], zarr_version=3
    ) as dataset:
        pos = dataset.create_position("0", "0", "0")
        pos["0"][:] = reference_data[np.newaxis, np.newaxis, ...]

    conversion_time = time.time() - t0

    # Read back
    t0 = time.time()
    with open_ome_zarr(zarr_v3_direct, mode="r") as dataset:
        pos = dataset["0/0/0"]
        zarr_data = np.array(pos["0"][0, 0, ...])
    read_time = time.time() - t0

    # Calculate metrics
    size_mb = get_size_mb(zarr_v3_direct)
    file_count = count_files(zarr_v3_direct)
    compression_ratio = reference_data.nbytes / (size_mb * 1024**2)
    ssim_val, psnr_val, mse_val = calculate_metrics(reference_data, zarr_data)

    results.append(
        {
            "conversion_path": "Original → OME-Zarr v3 (direct)",
            "format": "OME-Zarr v3",
            "conversion_time": conversion_time,
            "read_time": read_time,
            "size_mb": size_mb,
            "file_count": file_count,
            "compression_ratio": compression_ratio,
            "ssim": ssim_val,
            "psnr": psnr_val,
            "mse": mse_val,
            "success": True,
        }
    )

    print("✓ Success")
    print(f"  Conversion time: {conversion_time:.3f}s")
    print(f"  Read time: {read_time:.3f}s")
    print(f"  Size: {size_mb:.2f} MB ({compression_ratio:.2f}× compression)")
    print(f"  Files: {file_count}")
    print(f"  SSIM: {ssim_val:.6f}")

except Exception as e:
    print(f"✗ Failed: {e}")
    results.append(
        {
            "conversion_path": "Original → OME-Zarr v3 (direct)",
            "format": "OME-Zarr v3",
            "success": False,
            "error": str(e),
        }
    )


[5/5] Original → OME-Zarr v3 (direct)
✗ Failed: name 'output_dir' is not defined


NameError: name 'results' is not defined

## 3. Results Analysis

In [ ]:
# Create DataFrame
df = pd.DataFrame(results)
df_success = df[df["success"]].copy()

# Save to CSV
csv_path = output_dir / "conversion_benchmark_results.csv"
df.to_csv(csv_path, index=False)
print(f"✓ Saved results to: {csv_path}\n")

# Display results
if not df_success.empty:
    print("Successful Conversions:")
    display_cols = [
        "conversion_path",
        "conversion_time",
        "read_time",
        "size_mb",
        "file_count",
        "compression_ratio",
        "ssim",
    ]
    print(df_success[display_cols].to_string(index=False))
else:
    print("⚠️ No successful conversions")

if len(df[~df["success"]]) > 0:
    print("\nFailed Conversions:")
    print(df[~df["success"]][["conversion_path", "error"]].to_string(index=False))

In [ ]:
# Comparison analysis
if not df_success.empty:
    print("\n" + "=" * 70)
    print("CONVERSION PERFORMANCE COMPARISON")
    print("=" * 70)

    print("\n1. Conversion Speed:")
    for idx, row in df_success.iterrows():
        print(f"   {row['conversion_path']:40s}: {row['conversion_time']:6.3f}s")

    print("\n2. Read Speed:")
    for idx, row in df_success.iterrows():
        print(f"   {row['conversion_path']:40s}: {row['read_time']:6.3f}s")

    print("\n3. Storage Efficiency:")
    for idx, row in df_success.iterrows():
        print(
            f"   {row['conversion_path']:40s}: {row['size_mb']:6.2f} MB "
            f"({row['compression_ratio']:.2f}×)"
        )

    print("\n4. File Count:")
    for idx, row in df_success.iterrows():
        print(f"   {row['conversion_path']:40s}: {row['file_count']} files")

    print("\n5. Data Quality (SSIM):")
    for idx, row in df_success.iterrows():
        ssim_str = f"{row['ssim']:.6f}" if pd.notna(row["ssim"]) else "N/A"
        print(f"   {row['conversion_path']:40s}: {ssim_str}")

    print("\n" + "=" * 70)

## 4. Visualizations

In [ ]:
if not df_success.empty:
    fig, axes = plt.subplots(2, 3, figsize=(16, 10))
    fig.suptitle("iohub Format Conversion Benchmark", fontsize=14, fontweight="bold")

    paths = df_success["conversion_path"].tolist()
    x_pos = np.arange(len(paths))

    # Conversion time
    axes[0, 0].bar(x_pos, df_success["conversion_time"], color="steelblue")
    axes[0, 0].set_xticks(x_pos)
    axes[0, 0].set_xticklabels(paths, rotation=45, ha="right", fontsize=8)
    axes[0, 0].set_ylabel("Time (s)")
    axes[0, 0].set_title("Conversion Time")
    axes[0, 0].grid(True, alpha=0.3)

    # Read time
    axes[0, 1].bar(x_pos, df_success["read_time"], color="coral")
    axes[0, 1].set_xticks(x_pos)
    axes[0, 1].set_xticklabels(paths, rotation=45, ha="right", fontsize=8)
    axes[0, 1].set_ylabel("Time (s)")
    axes[0, 1].set_title("Read Time")
    axes[0, 1].grid(True, alpha=0.3)

    # Storage size
    axes[0, 2].bar(x_pos, df_success["size_mb"], color="green")
    axes[0, 2].set_xticks(x_pos)
    axes[0, 2].set_xticklabels(paths, rotation=45, ha="right", fontsize=8)
    axes[0, 2].set_ylabel("Size (MB)")
    axes[0, 2].set_title("Storage Size")
    axes[0, 2].grid(True, alpha=0.3)

    # Compression ratio
    axes[1, 0].bar(x_pos, df_success["compression_ratio"], color="purple")
    axes[1, 0].set_xticks(x_pos)
    axes[1, 0].set_xticklabels(paths, rotation=45, ha="right", fontsize=8)
    axes[1, 0].set_ylabel("Ratio")
    axes[1, 0].set_title("Compression Ratio")
    axes[1, 0].grid(True, alpha=0.3)

    # File count (log scale)
    axes[1, 1].bar(x_pos, df_success["file_count"], color="orange")
    axes[1, 1].set_xticks(x_pos)
    axes[1, 1].set_xticklabels(paths, rotation=45, ha="right", fontsize=8)
    axes[1, 1].set_ylabel("Count")
    axes[1, 1].set_title("File Count")
    axes[1, 1].set_yscale("log")
    axes[1, 1].grid(True, alpha=0.3)

    # SSIM
    axes[1, 2].bar(x_pos, df_success["ssim"], color="brown")
    axes[1, 2].set_xticks(x_pos)
    axes[1, 2].set_xticklabels(paths, rotation=45, ha="right", fontsize=8)
    axes[1, 2].set_ylabel("SSIM")
    axes[1, 2].set_title("Image Quality (SSIM)")
    axes[1, 2].set_ylim([0.999, 1.001])
    axes[1, 2].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig(output_dir / "conversion_benchmark.png", dpi=150, bbox_inches="tight")
    plt.show()

    print("✓ Visualizations complete")
else:
    print("⊘ No successful conversions to visualize")

## 5. Summary & Recommendations

In [ ]:
if not df_success.empty:
    print("=" * 70)
    print("IOHUB CONVERSION BENCHMARK SUMMARY")
    print("=" * 70)

    print("\n📊 Key Findings:")

    # Fastest conversion
    fastest = df_success.loc[df_success["conversion_time"].idxmin()]
    print(f"\n  Fastest conversion: {fastest['conversion_path']}")
    print(f"    Time: {fastest['conversion_time']:.3f}s")

    # Best compression
    best_comp = df_success.loc[df_success["compression_ratio"].idxmax()]
    print(f"\n  Best compression: {best_comp['conversion_path']}")
    print(f"    Ratio: {best_comp['compression_ratio']:.2f}×")
    print(f"    Size: {best_comp['size_mb']:.2f} MB")

    # Fewest files
    fewest = df_success.loc[df_success["file_count"].idxmin()]
    print(f"\n  Fewest files: {fewest['conversion_path']}")
    print(f"    Count: {fewest['file_count']}")

    print("\n💡 Recommendations:")
    print("\n  • For legacy data migration:")
    print("    - Use TIFF → OME-Zarr v2 for broad compatibility")
    print("    - Use TIFF → OME-Zarr v3 for modern pipelines")

    print("\n  • For new workflows:")
    print("    - Direct OME-Zarr conversion is faster (skips TIFF step)")
    print("    - v3 may offer better file management")

    print("\n  • Data quality:")
    print(
        f"    - All conversions maintain high fidelity (SSIM ≥ {df_success['ssim'].min():.4f})"
    )
    print("    - Lossless compression preserved across formats")

    print("\n🔗 Resources:")
    print("   • iohub: https://github.com/czbiohub-sf/iohub")
    print("   • OME-NGFF: https://ngff.openmicroscopy.org/")
    print("   • Zarr: https://zarr.readthedocs.io/")

    print("\n" + "=" * 70)
    print("✅ IOHUB CONVERSION BENCHMARK COMPLETE!")
    print("=" * 70)
else:
    print("⚠️ Benchmark completed but no successful conversions")